In [1]:
#imports
from dataset import Dataset
from feature_extractor import FeatureExtractor
from monitors_internals import MahalanobisMonitor, GaussianMixtureMonitor, OutsideTheBoxMonitor, MaxSoftmaxProbabilityMonitor,\
                    MaxLogitMonitor, EnergyMonitor, ReActMonitor
from monitors_input import SHINE_monitor, SHINE_monitor2
from evaluation import Evaluator
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, classification_report

import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
import torch
import utils

import os
import sys
import gzip
from PIL import Image
import scipy.io as spio

ModuleNotFoundError: No module named 'torchvision.models.feature_extraction'

In [ ]:
#model params
batch_size = 10
model = "resnet"

#monitor params
layer_relu_ids = [32]
additional_transform = None
adversarial_attack = None#

#dataset params
id_dataset = "cifar10"
ood_dataset = "svhn"
(id_X_train, id_y_train), (id_X_test, id_y_test) = utils.load_data(id_dataset, None)
(ood_X_train, ood_y_train), (ood_X_test, ood_y_test) = utils.load_data(ood_dataset, None)

In [ ]:
#loading data in pytorch way (for operations with ML models built with pytorch)

tensor_x = torch.Tensor(id_X_train) # transform to torch tensor
tensor_y = torch.Tensor(id_y_train)

my_trainset = TensorDataset(tensor_x,tensor_y)
my_trainset.name = id_dataset
my_trainset.split="train"
my_trainset.network=model
my_trainset.additional_transform=None
my_trainset.adversarial_attack=None
my_trainset.batch_size=batch_size

#test
tensor_x = torch.Tensor(id_X_test) # transform to torch tensor
tensor_y = torch.Tensor(id_y_test)
my_testset = TensorDataset(tensor_x,tensor_y)
my_testset.name = id_dataset
my_testset.split="test"
my_testset.network=model
my_testset.additional_transform=None
my_testset.adversarial_attack=None
my_testset.batch_size=batch_size

#ood
tensor_x = torch.Tensor(id_X_test) # transform to torch tensor
tensor_y = torch.Tensor(id_y_test)
my_dataset_ood = TensorDataset(tensor_x,tensor_y)
my_dataset_ood.name = ood_dataset
my_dataset_ood.split="test"
my_dataset_ood.network=model
my_dataset_ood.additional_transform=None
my_dataset_ood.adversarial_attack=None
my_dataset_ood.batch_size=batch_size

In [ ]:
#extracting and storing features from ID and OOD data using an ML model trained on ID data
feature_extractor = FeatureExtractor(model, id_dataset, layer_relu_ids)

features_train, logits_train, softmax_train, pred_train, lab_train = feature_extractor.get_features(my_trainset)
features_test, logits_test, softmax_test, pred_test, lab_test = feature_extractor.get_features(my_testset)
features_ood, logits_ood, softmax_ood, pred_ood, lab_ood = feature_extractor.get_features(my_dataset_ood)

In [ ]:
#accuracy of the ML model 
id_accuracy = accuracy_score(lab_test, pred_test)
ood_accuracy = 0
if id_dataset == ood_dataset:
    ood_accuracy = accuracy_score(lab_ood, pred_ood)

print("Accuracy")
print("ID:  ", id_accuracy)
print("OOD: ", ood_accuracy)

In [ ]:
id_layer_monitored = -1

In [ ]:
from scipy import spatial
import pickle
filename_c = 'Features/RESNET/cifar10_{}_correct.sav'
filename_i = 'Features/RESNET/cifar10_{}_incorrect.sav'

def similarity_act_func(features, id_y_train, pred_train):
    for cls in range(10):
        scores_1 = []
        scores_2 = []
        

        #all labels from class c
        ind_y_c = np.where(id_y_train == cls)[0]
        #print('len all labels class',c, np.shape(ind_y_c), ind_y_c)

        #all pred as c
        ind_ML_c = np.where(pred_train==cls)[0]
        #print('len pred class',c, np.shape(ind_ML_c), ind_ML_c)

        #features from correct pred
        ind = set(ind_y_c).intersection(ind_ML_c)
        f_c_correct = features[list(ind)]

        #features from incorrect pred
        ind = set(ind_y_c).symmetric_difference(ind_ML_c)
        f_c_incorrect = features[list(ind)]

        # how similar are act functions between themselves?
        for i in f_c_incorrect:
            for c in f_c_correct:
                cosine_similarity = 1 - spatial.distance.cosine(c, i)
                scores_1.append(cosine_similarity)

        for i in range(1,len(f_c_correct)):
            cosine_similarity = 1 - spatial.distance.cosine(f_c_correct[i-1], f_c_correct[i])
            scores_2.append(cosine_similarity)
            
        pickle.dump(scores_1, open(filename_i.format(cls), 'wb'))
        pickle.dump(scores_2, open(filename_c.format(cls), 'wb'))

        print('sim correct/incorrect pred', np.sum(scores_1)/len(scores_1))
        print('sim between pairs de correct pred', np.sum(scores_2)/len(scores_2))

In [ ]:
#training set
print('analysis for training set')
#features = features_train[id_layer_monitored]
#similarity_act_func(features, id_y_train, pred_train)

#test set
print('analysis for test set')
#features = features_test[id_layer_monitored]
#similarity_act_func(features, id_y_test, pred_test)

#OOD set
print('analysis for OOD test set')
#features = features_ood[id_layer_monitored]
#similarity_act_func(features, ood_y_test, pred_ood)

In [ ]:
#building SHINE monitor with ID data
#monitor_shine = SHINE_monitor(id_dataset)
#monitor_shine.fit_by_class_parallel(X_train, y_train)
#monitor_shine.fit_by_class(id_X_train, id_y_train, pred_train)

#print('number of monitors',len(monitor_shine.arr_density))


#building SHINE 2 monitor with ID data
#monitor_shine2 = SHINE_monitor2(id_dataset)
#monitor_shine.fit_by_class_parallel(X_train, y_train)
#monitor_shine2.fit_by_class(id_X_train, id_y_train, pred_train)

#print('number of monitors',len(monitor_shine2.arr_density))

In [ ]:
#same analysis but for SHINE scores

def shine_perf_analysis(id_X_train, id_y_training, pred_train, threshold_id):
    for c in range(10):
        scores_1 = []
        scores_2 = []
        #threshold_c = monitor_shine.scores_per_class[c]

        #all labels from class c
        ind_y_c = np.where(id_y_training == c)[0]
        #print('len all labels class',c, np.shape(ind_y_c), ind_y_c)

        #all pred as c
        ind_ML_c = np.where(pred_train==c)[0]
        #print('len pred class',c, np.shape(ind_ML_c), ind_ML_c)

        #images from correct pred
        ind = set(ind_y_c).intersection(ind_ML_c)
        X_c_correct = id_X_train[list(ind)]

        #scores from correct
        for x, pred in zip(X_c_correct, pred_train[list(ind)]):
            monitor_pred, pdf = monitor_shine.predict(np.array([x]), pred, threshold_id)
            scores_1.append(monitor_pred)

        #images from incorrect pred
        ind = set(ind_y_c).symmetric_difference(ind_ML_c)
        X_c_incorrect = id_X_train[list(ind)]

        #scores from incorrect
        for x, pred in zip(X_c_incorrect, pred_train[list(ind)]):
            monitor_pred, pdf = monitor_shine.predict(np.array([x]), pred, threshold_id)
            scores_2.append(monitor_pred)

        print('avg score for correct pred', 1-(np.sum(scores_1)/len(scores_1)))
        print('avg score for incorrect pred', 1-(np.sum(scores_2)/len(scores_2)))

In [ ]:
threshold_id = 0.9

#training set
print('training set analysis')
#shine_perf_analysis(id_X_train, id_y_train, pred_train, threshold_id)

#testing set
print('testing set analysis')
#shine_perf_analysis(id_X_test, id_y_test, pred_test, threshold_id)

#testing set
print('OOD test set analysis')
#shine_perf_analysis(ood_X_test, ood_y_test, pred_ood, threshold_id)

In [ ]:
#hybrid version of SHINE
arr_ood_threshold = {}
#arr_ood_threshold.update({0: 0.7043, 1: 0.7919, 2: 0.6996, 3: 0.7209, 4:0.7435, 5:0.7924, 6:0.6745, 7:0.7481,
#                         8:0.6934, 9:0.7831})
arr_ood_threshold.update({0: 0.7, 1: 0.7, 2: 0.6, 3: 0.7, 4:0.7, 5:0.7, 6:0.6, 7:0.7, 8:0.6, 9:0.7})

arr_id_threshold = {}
arr_id_threshold.update({0:0.9499, 1:0.9692, 2:0.9494, 3:0.9340, 4:0.9624, 5:0.9582, 6:0.9417, 7:0.9641,
                        8:0.9695, 9:0.9695})

#arr_id_threshold.update({0:0.94, 1:0.96, 2:0.94, 3:0.93, 4:0.96, 5:0.95, 6:0.94, 7:0.96, 8:0.96, 9:0.96})

def run_new_SHINE(monitor_shine, X, pred, incoming_feature, threshold_shine):
    features = features_train[id_layer_monitored]
    #all labels from class c
    ind_y_c = np.where(id_y_train == pred)[0]
    #print('len all labels class',c, np.shape(ind_y_c), ind_y_c)

    #all pred as c
    ind_ML_c = np.where(pred_train==pred)[0]
    #print('len pred class',c, np.shape(ind_ML_c), ind_ML_c)

    #features from correct pred
    ind = set(ind_y_c).intersection(ind_ML_c)
    f_c_correct = features[list(ind)]

    scores = []
    for c in f_c_correct:
        cosine_similarity = 1 - spatial.distance.cosine(c, incoming_feature)
        scores.append(cosine_similarity)
        
    avg_sim = np.sum(scores)/len(scores)
    
    if avg_sim >= arr_id_threshold[pred]:
        return False #it is ID and correct pred
    elif avg_sim < arr_ood_threshold[pred]:
        return True #it is OOD
    else:
        #it is not OOD but we do not know if the prediction is correct
        monitor_pred, pdf = monitor_shine.predict(X, pred, threshold_shine)
        return monitor_pred

In [ ]:
## experimenting new thresholds for shine
def run_SHINE_2(monitor_shine, X, pred, incoming_feature, threshold_SMimg, threshold_SMout):
    features = features_train[id_layer_monitored]
    #all labels from class c
    ind_y_c = np.where(id_y_train == pred)[0]
    #print('len all labels class',c, np.shape(ind_y_c), ind_y_c)

    #all pred as c
    ind_ML_c = np.where(pred_train==pred)[0]
    #print('len pred class',c, np.shape(ind_ML_c), ind_ML_c)

    #features from correct pred
    ind = set(ind_y_c).intersection(ind_ML_c)
    f_c_correct = features[list(ind)]

    scores = []
    for c in f_c_correct:
        cosine_similarity = 1 - spatial.distance.cosine(c, incoming_feature)
        scores.append(cosine_similarity)
        
    len_scores = len(scores)
    sorted_scores = sorted(scores)
    ind_threshold = int(len_scores*threshold_SMout)
    min_threshold_sim = sorted_scores[-ind_threshold]
    max_threshold_sim = sorted_scores[ind_threshold]
    #avg_sim = np.sum(scores)/len(scores)
    
    if max_threshold_sim >= arr_id_threshold[pred]:
        return False #it is ID and correct pred
    elif min_threshold_sim <= arr_ood_threshold[pred]:
        return True #it is OOD
    else:
        #it is not OOD but we do not know if the prediction is correct
        monitor_pred, pdf = monitor_shine.predict(X, pred, threshold_SMimg)
        return monitor_pred

In [ ]:
#building oob monitor
monitor_oob = OutsideTheBoxMonitor(n_clusters=3)
monitor_oob.fit(features_train[id_layer_monitored], lab_train)

In [ ]:
#building react monitor
monitor_react = ReActMonitor(quantile_value=0.99, mode='msp')
monitor_react.fit(feature_extractor, features_train[id_layer_monitored])

In [ ]:
# building softmax monitor
monitor_msp = MaxSoftmaxProbabilityMonitor()
monitor_msp.fit()

In [ ]:
# building max logit monitor
monitor_maxlogits = MaxLogitMonitor()
monitor_maxlogits.fit()

In [ ]:
# building Energy monitor
T = 1
monitor_energy = EnergyMonitor(temperature=T)
monitor_energy.fit()

In [ ]:
# building mahalanobis monitor
monitor_mahalanobis = MahalanobisMonitor(id_dataset, model, id_layer_monitored, is_tied=False)
monitor_mahalanobis.fit(features_train[id_layer_monitored], lab_train)

In [ ]:
#testing SHINE  and OOB monitors monitors on ID testset
m_true = []
m_shine = []
m_shine_2 = []
m_oob = []
m_react = []
m_msp = []
m_logits = []
m_energy = []
m_mhlnbis = []

threshold_SMimg = 0.1
threshold_SMout = 0.9

for x, pred, feature, softmax, logits, label in tqdm(zip(id_X_test, pred_test, features_test[id_layer_monitored],
                                                 softmax_test, logits_test, id_y_test.flatten())):
    #SHINE
    #monitor_shine_pred = run_new_SHINE(monitor_shine2, np.array([x]), pred, feature, threshold_id)
    #m_shine.append(monitor_shine_pred)
    
    # SHINE relaxed thresholds
    #monitor_shine_pred_2 = run_SHINE_2(monitor_shine2, np.array([x]), pred, feature,
    #                                   threshold_SMimg, threshold_SMout)
    #m_shine_2.append(monitor_shine_pred_2)
    
    #oob
    out_box = monitor_oob.predict([feature], [pred])
    m_oob.append(out_box)
    
    #react
    r = monitor_react.predict([feature])
    m_react.append(r)
    
    #Max Softmax Probability
    msp = monitor_msp.predict([softmax])
    m_msp.append(msp)
    
    #Max logit
    maxlogits = monitor_maxlogits.predict([logits])
    m_logits.append(maxlogits)
    
    #Energy
    energy = monitor_energy.predict(logits)
    m_energy.append(energy)
    
    #mahalanobis
    #mahalanobis = monitor_mahalanobis.predict(np.array([feature]), [pred])
    #m_mhlnbis.append(mahalanobis)
    
    if pred == label: #monitor does not need to activate
        m_true.append(False)
    else: #monitor should activate
        m_true.append(True)

In [ ]:
for x, feature, softmax, logits, pred in tqdm(zip(ood_X_test, features_ood[id_layer_monitored], softmax_ood,
                                          logits_ood, pred_ood)):
    #SHINE
    #monitor_shine_pred = run_new_SHINE(monitor_shine2, np.array([x]), pred, feature, threshold_id)
    #m_shine.append(monitor_shine_pred)
    
    # SHINE relaxed thresholds
    #monitor_shine_pred_2 = run_SHINE_2(monitor_shine2, np.array([x]), pred, feature,
    #                                   threshold_SMimg, threshold_SMout)
    #m_shine_2.append(monitor_shine_pred_2)
    
    #oob
    out_box = monitor_oob.predict([feature], [pred])
    m_oob.append(out_box)
    
    #react
    r = monitor_react.predict([feature])
    m_react.append(r)
    
    #Max Softmax Probability
    msp = monitor_msp.predict([softmax])
    m_msp.append(msp)
    
    #Max logit
    maxlogits = monitor_maxlogits.predict([logits])
    m_logits.append(maxlogits)
    
    #Energy
    energy = monitor_energy.predict(logits)
    m_energy.append(energy)
    
    #mahalanobis
    #mahalanobis = monitor_mahalanobis.predict([feature], [pred])
    #m_mhlnbis.append(mahalanobis)
    
    m_true.append(True) #monitor should always react to novel classes

In [ ]:
from sklearn.metrics import matthews_corrcoef as mcc
from sklearn.metrics import balanced_accuracy_score

m_true = np.array(m_true).astype(bool)
#m_shine = np.array(m_shine).astype(bool)
#m_shine_2 = np.array(m_shine_2).astype(bool)
m_oob = np.array(m_oob).astype(bool)
m_react = np.array(m_react).astype(bool)
m_msp = np.array(m_msp).astype(bool)
m_logits = np.array(m_logits).astype(bool)
m_energy = np.array(m_energy).astype(bool)
m_mhlnbis = np.array(m_mhlnbis).astype(bool)

#evaluating new SHINE 
#print('\nSHINE new')
#print(classification_report(m_true, m_shine))
#print(mcc(m_true, m_shine))
#print(balanced_accuracy_score(m_true, m_shine))

#evaluating new SHINE 2
#print('\nSHINE new 2')
#print(classification_report(m_true, m_shine_2))
#print(mcc(m_true, m_shine_2))
#print(balanced_accuracy_score(m_true, m_shine_2))

#evaluating OOB
print('\nOOB')
print(classification_report(m_true, m_oob))
print(mcc(m_true, m_oob))
print(balanced_accuracy_score(m_true, m_oob))

#evaluating React
print('\nReact')
print(classification_report(m_true, m_react))
print(mcc(m_true, m_react))
print(balanced_accuracy_score(m_true, m_react))

#evaluating Max Softmax probability
print('\nMax Softmax probability')
print(classification_report(m_true, m_msp))
print(mcc(m_true, m_msp))
print(balanced_accuracy_score(m_true, m_msp))

#evaluating Max Logits
print('\nMax Logits')
print(classification_report(m_true, m_logits))
print(mcc(m_true, m_logits))
print(balanced_accuracy_score(m_true, m_logits))

#evaluating Energy
print('\nEnergy')
print(classification_report(m_true, m_energy))
print(mcc(m_true, m_energy))
print(balanced_accuracy_score(m_true, m_energy))

#evaluating Mahalanobis
#print('\nMahalanobis')
#print(classification_report(m_true, m_mhlnbis))
#print(mcc(m_true, m_mhlnbis))
#print(balanced_accuracy_score(m_true, m_mhlnbis))

In [ ]:
'''
id_threshold = int(len_scores*threshold_shine)
min_threshold_sim = sorted_scores[-id_threshold]
max_threshold_sim = sorted_scores[id_threshold]

SHINE new 2
              precision    recall  f1-score   support

       False       0.62      0.86      0.72      9367
        True       0.94      0.82      0.87     26665

    accuracy                           0.83     36032
   macro avg       0.78      0.84      0.80     36032
weighted avg       0.86      0.83      0.83     36032

0.6156203195994662
0.8364962945766474
'''